In [12]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os as os

########################################################################################

folder = r"C:\Users\op354\OneDrive\Documentos\Oscar\UG\Repositorio Github\ejerciosytareas-Oscarraul28\Img\CatyDog"
def imagenes(folder, dim=(64,64)):
    X, Y = [], []
    for filename in os.listdir(folder):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")): #usamos el lower para evitar confusiones con mayusculas
            ruta = os.path.join(folder, filename)
            #Aqui tratamos las imagenes como ya se habia hecho:
            img = Image.open(ruta)
            img = img.resize(dim)               
            arr = np.array(img) / 255.0 
            #Vectorizamos y metemos en el arreglo X:
            X.append(arr.flatten()) 
            #Ahora llenamos el arreglo Y, para ello clasificamos segun el nombre de la imagen:
            if filename.startswith("dog"):
                Y.append(0)  
            elif filename.startswith("cat"):
                Y.append(1) 
    return np.array(X).T, np.array(Y).reshape(1, -1)

X,Y = imagenes(folder, dim=(64,64))


########################################################################################

#Ahora si procedemos a inicializar los datos, usamos aleatoriedad para ayudar al aprendizaje de nuestro modelo:

np.random.seed(1)                       #Con esto fijamos un numero aleatorio para evitar que cambie cada que se corra el codigo
n_x = X.shape[0]                        #Para extraer los pixeles que tiene cada imagen
w = np.random.randn(n_x, 1) * 0.01      #Le doy numeros chicos para evitar sobrecarga en el calculo
b = 0
a= 0.1                                  # (es la tasa de aprendizaje)
iteraciones = 1000                                # numero de iteraciones

#Definimos una funcion para nuestra funcion sigmoide:
def sigmoide(z):
    return 1 / (1 + np.exp(-z))

#Definimos otra funcion para calcular el costo, dw y db que nos ayudaran a hacer las backward y forward propagations
def propagaciones(w, b, X, Y):
    m = X.shape[1]
    # Forward
    Z = np.dot(w.T, X) + b
    A = sigmoide(Z)                      #Este es nuestro vector de predicciones "Y con gorrito"
    costo = -(1/m) * np.sum(Y*np.log(A+1e-8) + (1-Y)*np.log(1-A+1e-8))   #el 1e-8 es para evitar errores numericos en caso de tener 0 o 1
    # Backward
    dZ = A - Y
    dw = (1/m) * np.dot(X, dZ.T)
    db = (1/m) * np.sum(dZ)
    return dw, db, costo

def mejoramiento(w, b, X, Y, iteraciones, a):
    for i in range(iteraciones):
        dw, db, costo = propagaciones(w, b, X, Y)
        # Gradient descent
        w -= a * dw
        b -= a * db
        #Para ir viendo si nuestro si nuestro costo va bajando con el entrenamiento aplicamos:
        if i % 200 == 0:
            print(f"En la iteracion numero {i}, Tenemos un costo de: {costo:.4f}")
    return w, b

#Ahora si viene el entrenamiento:
w, b = mejoramiento(w, b, X, Y, iteraciones, a)

#Y finalmente:
def predicciones(X, w, b):
    A = sigmoide(np.dot(w.T, X) + b)
    return (A > 0.5).astype(int)        #Aqui aplicamos el umbral de desicion y convertimos el True/False en 0/1

def cambiar(ruta, dim=(64,64)):
    img = Image.open(ruta)
    img = img.resize(dim)
    arr = np.array(img) / 255.0
    return arr.flatten().reshape(-1,1)

#Vemos que tanto coincide:
predicc = predicciones(X, w, b)
print("Predicciones:", predicc)
print("Etiquetas reales:", Y)

precision = np.mean(predicc == Y) * 100
print(f"La precisión en el entrenamiento es: {precision:.2f}%")

#Ahora si probamos el algoritmo con una imagen nueva, recordando que perros=0 y gatos=1:
nueva = cambiar("Img/gatonuevo1.jpg")
pred_nueva = predicciones(nueva, w, b)
print("Predicción de nueva imagen:", pred_nueva)


En la iteracion numero 0, Tenemos un costo de: 0.7778
En la iteracion numero 200, Tenemos un costo de: 4.7342
En la iteracion numero 400, Tenemos un costo de: 0.0031
En la iteracion numero 600, Tenemos un costo de: 0.0010
En la iteracion numero 800, Tenemos un costo de: 0.0006
Predicciones: [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]]
Etiquetas reales: [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]]
La precisión en el entrenamiento es: 100.00%
Predicción de nueva imagen: [[1]]
